In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Load environment variables
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    raise ValueError("No API key found. Please set the OPENAI_API_KEY in your .env file.")
elif not api_key.startswith("sk-proj-"):
    raise ValueError("Invalid API key format. Make sure it starts with 'sk-proj-'.")
elif api_key.strip() != api_key:
    raise ValueError("API key contains extra spaces. Please fix your .env file.")

# Initialize OpenAI
openai = OpenAI(api_key=api_key)

# Headers for web requests
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        self.title = ""
        self.text = ""
        self._fetch_website_content()

    def _fetch_website_content(self):
        try:
            response = requests.get(self.url, headers=HEADERS)
            response.raise_for_status()  # Raise an error for HTTP codes 4xx/5xx
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Extract website title and clean text
            self.title = soup.title.string if soup.title else "No title found"
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        except Exception as e:
            raise RuntimeError(f"Error fetching website content: {e}")

# Prompts
system_prompt = """
You are a social media assistant skilled in creating tailored, engaging posts for Facebook, Instagram, and X. \
Generate 2 optimized posts for each platform based on the provided blog post or landingpage. \
One of them should be a catchy one-liner. \
Follow the specific tone, guidelines, and character limits for each platform:
- Facebook: Engaging and community-oriented, up to 280 characters.
- Instagram: Visual and hashtag-driven, up to 2200 characters.
- X (formerly Twitter): Concise and attention-grabbing, up to 280 characters. 
"""

def user_prompt_for(website):
    return (
        f"The website title is '{website.title}'.\n"
        f"Here is the content of the page:\n{website.text}\n\n"
        "Based on this content, create engaging social media posts tailored to the following platforms:\n"
        "- Facebook: Include a compelling hook and a call-to-action.\n"
        "- Instagram: Focus on visual storytelling and include relevant hashtags.\n"
        "- X: Write concise, attention-grabbing posts.\n"
        "Ensure each post resonates with the target audience and aligns with platform best practices."
    )


def create_messages(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

def generate_social_posts(url):
    website = Website(url)
    messages = create_messages(website)
    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        raise RuntimeError(f"Error generating social posts: {e}")

def display_social_posts(url):
    try:
        posts = generate_social_posts(url)
        display(Markdown(posts))
    except Exception as e:
        print(f"Error: {e}")

# Generate and display posts
display_social_posts("www.your-content.com")